In [1]:
import xtrack as xt
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import apertls

In [21]:
path_lines = Path('/Users/gebruiker/sps_simulations/injection_lines')
line = xt.Line.from_json(Path(path_lines, 'sps_with_aperture_inj_q20.json'))
#line = xt.Line.from_json(Path(path_lines, 'injection_thin_approx_ap_with_deferred_expressions.json'))
#line = xt.Line.from_json(Path(path_lines, 'sps_lhc_q20_rf_with_ap.json'))
#Set cavitiy voltages correctly
line['acl.31735'].frequency = 800e6
line['acl.31735'].voltage = 2 * 0.18e6
line['acl.31735'].lag = 180

line['actcse.31632'].frequency = 200e6
line['actcse.31632'].voltage = 4.5e6
line['actcse.31632'].lag = 180

tw_ref = line.twiss4d()

for name in line.element_names:
    if line[name].__class__.__name__.startswith('Limit') or line[name].__class__.__name__.startswith('Multipole'):
        #Dogleg in LSS5 for extraction
        if 'qd.51710' in name:
            line[name].shift_y = -5.273e-3
        elif 'qfa.51810' in name:
            line[name].shift_y = -17.369e-3
        elif 'qd.51910' in name:
            line[name].shift_y = -5.273e-3
        #Dogleg in LSS1 for injection
        elif 'qd.11710' in name:
            line[name].shift_x = -4.80e-3
        elif 'qfa.11810' in name:
            line[name].shift_x = -2.97e-3
        elif 'qda.11910' in name:
            line[name].shift_x = -4.80e-3
    
    elif line[name].__class__.__name__.startswith('ThinSliceQuadrupole') or line[name].__class__.__name__.startswith('Quadrupole'):
        line['qd.51710'].shift_y = -5.273e-3
        line['qfa.51810'].shift_y = -17.369e-3
        line['qd.51910'].shift_y = -5.273e-3
        line['qd.11710'].shift_x = -4.80e-3 #Hannes Thesis says qda.
        line['qf.11810'].shift_x = -2.97e-3 #Hannes Thesis says qfa.
        line['qda.11910'].shift_x = -4.80e-3

tw_before = line.twiss4d()
tw = line.twiss()

Loading line from dict:   0%|          | 0/17781 [00:00<?, ?it/s]

Done loading line from dict.           


In [14]:
tt = line.get_table()

#Orbit correction
tt_monitors = tt.rows['bp.*'].rows['.*(?<!_entry)$'].rows['.*(?<!_exit)$']
line.steering_monitors_x = tt_monitors.name
line.steering_monitors_y = tt_monitors.name

tt_h_correctors = tt.rows['mdh\..*'].rows['.*h\..*']
mask_ap_h = np.array([el.startswith('Limit') for el in tt_h_correctors.element_type])
line.steering_correctors_x = tt_h_correctors.name[~mask_ap_h]

tt_v_correctors = tt.rows['mdv\..*'].rows['.*v\..*']
mask_ap_v = np.array([el.startswith('Limit') for el in tt_v_correctors.element_type])
line.steering_correctors_y = tt_v_correctors.name[~mask_ap_v]

orbit_correction = line.correct_trajectory(twiss_table=tw_ref,n_micado=5, n_iter=10)

#Plot orbits to verify
tw = line.twiss()
fig, ax = plt.subplots(1,2, figsize=(10,4))
ax[0].plot(tw.s, tw.x)
ax[0].set_title('Closed orbit x after orbit correction')
ax[0].grid()
ax[1].plot(tw.s, tw.y)
ax[1].set_title('Closed orbit y after orbit correction')
ax[1].grid()

<>:8: SyntaxWarning: invalid escape sequence '\.'
<>:8: SyntaxWarning: invalid escape sequence '\.'
<>:12: SyntaxWarning: invalid escape sequence '\.'
<>:12: SyntaxWarning: invalid escape sequence '\.'
<>:8: SyntaxWarning: invalid escape sequence '\.'
<>:8: SyntaxWarning: invalid escape sequence '\.'
<>:12: SyntaxWarning: invalid escape sequence '\.'
<>:12: SyntaxWarning: invalid escape sequence '\.'
/var/folders/q4/rcyb5f2524q8p03j88z0mxj80000gn/T/ipykernel_77789/2267181951.py:8: SyntaxWarning: invalid escape sequence '\.'
  tt_h_correctors = tt.rows['mdh\..*'].rows['.*h\..*']
/var/folders/q4/rcyb5f2524q8p03j88z0mxj80000gn/T/ipykernel_77789/2267181951.py:8: SyntaxWarning: invalid escape sequence '\.'
  tt_h_correctors = tt.rows['mdh\..*'].rows['.*h\..*']
/var/folders/q4/rcyb5f2524q8p03j88z0mxj80000gn/T/ipykernel_77789/2267181951.py:12: SyntaxWarning: invalid escape sequence '\.'
  tt_v_correctors = tt.rows['mdv\..*'].rows['.*v\..*']
/var/folders/q4/rcyb5f2524q8p03j88z0mxj80000gn/T/ipy

Iteration 0, x_rms: 3.97e-03 -> 1.21e-04, y_rms: 1.54e-03 -> 2.85e-04
Iteration 1, x_rms: 1.21e-04 -> 1.21e-04, y_rms: 2.85e-04 -> 2.81e-04
Iteration 2, x_rms: 1.21e-04 -> 1.21e-04, y_rms: 2.81e-04 -> 2.79e-04
Iteration 3, x_rms: 1.21e-04 -> 1.21e-04, y_rms: 2.79e-04 -> 2.78e-04
Iteration 4, x_rms: 1.21e-04 -> 1.21e-04, y_rms: 2.78e-04 -> 2.77e-04
Iteration 5, x_rms: 1.21e-04 -> 1.21e-04, y_rms: 2.77e-04 -> 2.77e-04
Iteration 6, x_rms: 1.21e-04 -> 1.21e-04, y_rms: 2.77e-04 -> 2.77e-04
Iteration 7, x_rms: 1.21e-04 -> 1.21e-04, y_rms: 2.77e-04 -> 2.77e-04
Iteration 8, x_rms: 1.21e-04 -> 1.21e-04, y_rms: 2.77e-04 -> 2.77e-04
Iteration 9, x_rms: 1.21e-04 -> 1.21e-04, y_rms: 2.77e-04 -> 2.77e-04


In [4]:
def circle(radius, x_0, y_0, t):
    x = x_0 + radius*np.cos(t)
    y = y_0 + radius*np.sin(t)

    return x, y

def circle_vertical_coord(radius, x_0, y_0, x):
    t = np.acos((x-x_0)/radius)
    y = y_0 + radius*np.sin(t)

    return y
    
def straight_line(alpha, x_0, y_0, x):
    return alpha*(x-x_0) + y_0

def get_sagitta_effect_on_aperture(s, length, bending_radius, offset_middle_dipole, s_start_dipole = 0):
    #center of the bending circle
    s0 = s_start_dipole + length/2 
    x0 = -(bending_radius-offset_middle_dipole)

    #getting x position for that circle
    x = circle_vertical_coord(bending_radius, s0, x0, s)

    return x

In [5]:
# #Apertures for each dipole type
# aper_mbb = line['vebb.10070.b_aper'].copy()
# aper_mba = line['veba.10030.b_aper'].copy()

# #Defining bending radius
# R_mba = 1/line['mba.10030'].k0
# R_mbb = 1/line['mbb.10070'].k0
# assert R_mba == R_mbb
# bending_radius = 1/line['mba.10030'].k0

# #Defining magnetic length which is considered as the length of the magnet
# assert line['mba.10030'].length == line['mbb.10070'].length
# length = line['mba.10030'].length

# #Sagitta offsets (predefined)
# offset_end_dipole=4.4e-3
# offset_middle_dipole=2.2e-3

In [6]:
def make_non_replica_element(ee):
    # I did not add rotation because sps has no rotated elements
    if ee.__class__.__name__ == 'LimitEllipse':
        return xt.LimitEllipse(a=ee.a, b=ee.b, shift_x=ee.shift_x, shift_y=ee.shift_y)

    elif ee.__class__.__name__ == 'LimitRect':
        return xt.LimitRect(min_x = ee.min_x, max_x = ee.max_x, min_y = ee.min_y, max_y = ee.max_y, shift_x = ee.shift_x, shift_y = ee.shift_y)
    
    elif ee.__class__.__name__ == 'LimitRectEllipse':
        return xt.LimitRectEllipse(a=ee.a, b=ee.b, max_x = ee.max_x, max_y = ee.max_y, shift_x = ee.shift_x, shift_y = ee.shift_y)
    
    elif ee.__class__.__name__ == 'LimitRacetrack':
        return xt.LimitRacetrack(a=ee.a, b=ee.b, min_x = ee.min_x, max_x = ee.max_x, min_y = ee.min_y, max_y = ee.max_y, shift_x = ee.shift_x, shift_y = ee.shift_y)
    else:
        AssertionError('Aperture type is not known')

In [5]:
def make_sagitta_dipole_apertures(x_orbits, aper_element):
    aperture_slices = []

    for x_pos in x_orbits:
        aper_pos = aper_element.copy()
        #aper_pos = make_non_replica_element(aper_element)
        aper_pos.shift_x = -x_pos
        aperture_slices.append(aper_pos)
    
    return np.array(aperture_slices)

def generate_aperture_slices_names(name_element, nb_slices):
    slices_names = []

    for i in range(nb_slices):
        slices_names.append(f'{name_element}..{i}_aper2')
        #slices_names.append(f'{name_element}_aper..{i}')
    
    return slices_names

In [8]:
# env = xt.Environment()
# line_test_mbb = env.new_line(components=[
#     env.new('aper_upstream', aper_mbb.__class__, min_x = aper_mbb.min_x, max_x = aper_mbb.max_x, min_y = aper_mbb.min_y, max_y = aper_mbb.max_y),
#     env.new('bend_test', xt.RBend, length = line['mbb.10070'].length, k0=line['mbb.10070'].k0),
#     env.new('aper_downstream', aper_mbb.__class__, min_x = aper_mbb.min_x, max_x = aper_mbb.max_x, min_y = aper_mbb.min_y, max_y = aper_mbb.max_y)])
# line_test_mbb.particle_ref = line.particle_ref.copy()

# nb_slices = 11
# s_start = line_test_mbb.get_s_position('bend_test')
# s_stop = s_start + line_test_mbb['bend_test'].length
# s_positions = np.linspace(s_start, s_stop, nb_slices)

# x_orbits = get_sagitta_effect_on_aperture(s_positions, length, bending_radius, offset_middle_dipole)
# aper_slices = make_sagitta_dipole_apertures(x_orbits, aper_mbb)
# slices_names = generate_aperture_slices_names('mbb.10070', nb_slices)

# insertions = []
# env = line_test_mbb.env
# for nn, ee, ss in zip(slices_names, aper_slices, s_positions):
#     env.elements[nn] = ee
#     insertions.append(env.place(nn, at=ss))
# line_test_mbb.insert(insertions)

In [9]:
%matplotlib tk
line_thick = xt.Line.from_json('../../injection_lines/injection_thick_approx_ap.json')
plotter_before = apertls.InteractiveAperturePlotter(line, line_thick)

Loading line from dict:   0%|          | 0/7477 [00:00<?, ?it/s]

Done loading line from dict.           


2025-04-18 18:12:01.911 python[75405:6322020] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-18 18:12:01.911 python[75405:6322020] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [10]:
# env = line.env

# nb_slices = 11


# #Information dipoles
# #Apertures for each dipole type
# aper_mbb = line['vebb.10070.b_aper'].copy()
# aper_mba = line['veba.10030.b_aper'].copy()

# #Defining bending radius
# R_mba = 1/line['mba.10030'].k0
# R_mbb = 1/line['mbb.10070'].k0
# assert R_mba == R_mbb
# bending_radius = 1/line['mba.10030'].k0

# #Defining magnetic length which is considered as the length of the magnet
# assert line['mba.10030'].length == line['mbb.10070'].length
# length = line['mba.10030'].length

# #Sagitta offsets (predefined)
# offset_end_dipole=4.4e-3
# offset_middle_dipole=2.2e-3

# #mba magnets
# names_mba = tt.rows['mba.*'].name
# positions_mba = tt.rows['mba.*'].s
# s_positions_generic_mba = np.linspace(0, length, nb_slices)
# x_orbits_mba = get_sagitta_effect_on_aperture(s_positions_generic_mba, length, bending_radius, offset_middle_dipole)
# aper_slices_mba = make_sagitta_dipole_apertures(x_orbits_mba, aper_mba)

# #mbb magnets
# names_mbb = tt.rows['mbb.*'].name
# positions_mbb = tt.rows['mbb.*'].s
# s_positions_generic_mbb = np.linspace(0, length, nb_slices)
# x_orbits_mbb = get_sagitta_effect_on_aperture(s_positions_generic_mbb, length, bending_radius, offset_middle_dipole)
# aper_slices_mbb = make_sagitta_dipole_apertures(x_orbits_mbb, aper_mbb)

# insertions = []

# for i in range(len(names_mba)):
#     s_positions = np.linspace(positions_mba[i], positions_mba[i] + length, nb_slices)
#     aperture_slices_names = generate_aperture_slices_names(names_mba[i], nb_slices)
    
#     for nn, ee, ss in zip(aperture_slices_names, aper_slices_mba, s_positions):
#         # #env.elements[nn] = ee.copy()
#         # env.elements[nn] = make_non_replica_element(ee)
#         # insertions.append(env.place(nn, at=ss))

#         if ee.__class__.__name__ == 'LimitEllipse':
#             env.new(nn, xt.LimitEllipse, a=ee.a, b=ee.b, shift_x=ee.shift_x, shift_y=ee.shift_y)
#         elif ee.__class__.__name__ == 'LimitRect':
#             env.new(nn, xt.LimitRect, min_x=ee.min_x, max_x=ee.max_x, min_y=ee.min_y, max_y=ee.max_y, shift_x=ee.shift_x, shift_y=ee.shift_y)
#         elif ee.__class__.__name__ == 'LimitRectEllipse':
#             env.new(nn, xt.RectEllipse, a=ee.a, b=ee.b, max_x=ee.max_x, max_y=ee.max_y, shift_x=ee.shift_x, shift_y=ee.shift_y)
#         elif ee.__class__.__name__ == 'LimitRacetrack':
#             env.new(nn, xt.LimitRacetrack, a=ee.a, b=ee.b, min_x=ee.min_x, max_x=ee.max_x, min_y=ee.min_y, max_y=ee.max_y, shift_x=ee.shift_x, shift_y=ee.shift_y)
#         insertions.append(env.place(nn, at=ss))


# for i in range(len(names_mbb)):
#     s_positions = np.linspace(positions_mbb[i], positions_mbb[i] + length, nb_slices)
#     aperture_slices_names = generate_aperture_slices_names(names_mbb[i], nb_slices)
    
#     for nn, ee, ss in zip(aperture_slices_names, aper_slices_mbb, s_positions):
#         # #env.elements[nn] = ee.copy()
#         # env.elements[nn] = make_non_replica_element(ee)
#         # insertions.append(env.place(nn, at=ss))
#         if ee.__class__.__name__ == 'LimitEllipse':
#             env.new(nn, xt.LimitEllipse, a=ee.a, b=ee.b, shift_x=ee.shift_x, shift_y=ee.shift_y)
#         elif ee.__class__.__name__ == 'LimitRect':
#             env.new(nn, xt.LimitRect, min_x=ee.min_x, max_x=ee.max_x, min_y=ee.min_y, max_y=ee.max_y, shift_x=ee.shift_x, shift_y=ee.shift_y)
#         elif ee.__class__.__name__ == 'LimitRectEllipse':
#             env.new(nn, xt.RectEllipse, a=ee.a, b=ee.b, max_x=ee.max_x, max_y=ee.max_y, shift_x=ee.shift_x, shift_y=ee.shift_y)
#         elif ee.__class__.__name__ == 'LimitRacetrack':
#             env.new(nn, xt.LimitRacetrack, a=ee.a, b=ee.b, min_x=ee.min_x, max_x=ee.max_x, min_y=ee.min_y, max_y=ee.max_y, shift_x=ee.shift_x, shift_y=ee.shift_y)
#         insertions.append(env.place(nn, at=ss))

# line.insert(insertions)

In [6]:
env = line.env

nb_slices = 11

#Defining bending radius
R_mba = 1/line['mba.10030'].k0
R_mbb = 1/line['mbb.10070'].k0
assert R_mba == R_mbb
bending_radius = 1/line['mba.10030'].k0

#Defining magnetic length which is considered as the length of the magnet
assert line['mba.10030'].length == line['mbb.10070'].length
length = line['mba.10030'].length

#Sagitta offsets (predefined)
offset_end_dipole=4.4e-3
offset_middle_dipole=2.2e-3

#mba magnets
names_mba = tt.rows['mba.*'].name
positions_mba = tt.rows['mba.*'].s
s_positions_generic_mba = np.linspace(0, length, nb_slices)
x_orbits_mba = get_sagitta_effect_on_aperture(s_positions_generic_mba, length, bending_radius, offset_middle_dipole)
dim_x_mba = 0.076
dim_y_mba = 0.0193
# mba_sagitta_apertures = [xt.LimitRect(min_x=-dim_x_mba, max_x=dim_x_mba, min_y=-dim_y_mba, max_y=dim_y_mba, shift_x=-x_orbits_mba[i], shift_y=0) for i in range(len(x_orbits_mba))]
mba_sagitta_apertures = [xt.LimitRect(min_x=-dim_x_mba-x_orbits_mba[i], max_x=dim_x_mba-x_orbits_mba[i], min_y=-dim_y_mba, max_y=dim_y_mba) for i in range(len(x_orbits_mba))]

#mbb magnets
names_mbb = tt.rows['mbb.*'].name
positions_mbb = tt.rows['mbb.*'].s
s_positions_generic_mbb = np.linspace(0, length, nb_slices)
x_orbits_mbb = get_sagitta_effect_on_aperture(s_positions_generic_mbb, length, bending_radius, offset_middle_dipole)
dim_x_mbb = 0.0645
dim_y_mbb = 0.0267
# mbb_sagitta_apertures = [xt.LimitRect(min_x=-dim_x_mbb, max_x=dim_x_mbb, min_y=-dim_y_mbb, max_y=dim_y_mbb, shift_x=-x_orbits_mbb[i], shift_y=0) for i in range(len(x_orbits_mbb))]
mbb_sagitta_apertures = [xt.LimitRect(min_x=-dim_x_mbb-x_orbits_mbb[i], max_x=dim_x_mbb-x_orbits_mbb[i], min_y=-dim_y_mbb, max_y=dim_y_mbb) for i in range(len(x_orbits_mbb))]

insertions = []

for i in range(len(names_mba)):
    s_positions = np.linspace(positions_mba[i], positions_mba[i] + length, nb_slices)
    aperture_slices_names = generate_aperture_slices_names(names_mba[i], nb_slices)
    
    for nn, ee, ss in zip(aperture_slices_names, mba_sagitta_apertures, s_positions):
        env.elements[nn] = ee.copy()
        insertions.append(env.place(nn, at=ss))


for i in range(len(names_mbb)):
    s_positions = np.linspace(positions_mbb[i], positions_mbb[i] + length, nb_slices)
    aperture_slices_names = generate_aperture_slices_names(names_mbb[i], nb_slices)
    
    for nn, ee, ss in zip(aperture_slices_names, mbb_sagitta_apertures, s_positions):
        s_positions = np.linspace(positions_mbb[i], positions_mbb[i] + length, nb_slices)
        aperture_slices_names = generate_aperture_slices_names(names_mbb[i], nb_slices)
        
        for nn, ee, ss in zip(aperture_slices_names, mbb_sagitta_apertures, s_positions):
            env.elements[nn] = ee.copy()
            insertions.append(env.place(nn, at=ss))

line.insert(insertions)

Slicing line:   0%|          | 0/15184 [00:00<?, ?it/s]

In [12]:
# env = line.env

# nb_slices = 11


# #Information dipoles
# #Apertures for each dipole type
# aper_mbb = line['vebb.10070.b_aper'].copy()
# aper_mba = line['veba.10030.b_aper'].copy()

# #Defining bending radius
# R_mba = 1/line['mba.10030'].k0
# R_mbb = 1/line['mbb.10070'].k0
# assert R_mba == R_mbb
# bending_radius = 1/line['mba.10030'].k0

# #Defining magnetic length which is considered as the length of the magnet
# assert line['mba.10030'].length == line['mbb.10070'].length
# length = line['mba.10030'].length

# #Sagitta offsets (predefined)
# offset_end_dipole=4.4e-3
# offset_middle_dipole=2.2e-3

# #mba magnets
# names_mba = tt.rows['mba.*'].name
# positions_mba = tt.rows['mba.*'].s
# s_positions_generic_mba = np.linspace(0, length, nb_slices)
# x_orbits_mba = get_sagitta_effect_on_aperture(s_positions_generic_mba, length, bending_radius, offset_middle_dipole)
# aper_slices_mba = make_sagitta_dipole_apertures(x_orbits_mba, aper_mba)

# #mbb magnets
# names_mbb = tt.rows['mbb.*'].name
# positions_mbb = tt.rows['mbb.*'].s
# s_positions_generic_mbb = np.linspace(0, length, nb_slices)
# x_orbits_mbb = get_sagitta_effect_on_aperture(s_positions_generic_mbb, length, bending_radius, offset_middle_dipole)
# aper_slices_mbb = make_sagitta_dipole_apertures(x_orbits_mbb, aper_mbb)

# insertions = []

# for i in range(len(names_mba)):
#     s_positions = np.linspace(positions_mba[i], positions_mba[i] + length, nb_slices)
#     x_orbits = get_sagitta_effect_on_aperture(s_positions, length, bending_radius, offset_middle_dipole, s_start_dipole=positions_mba[0])
#     aperture_slices_names = generate_aperture_slices_names(names_mba[i], nb_slices)
#     aper_slices = make_sagitta_dipole_apertures(x_orbits, aper_mba)
    
#     for nn, ee, ss in zip(aperture_slices_names, aper_slices, s_positions):
#         env.elements[nn] = ee.copy()
#         insertions.append(env.place(nn, at=ss))


# for i in range(len(names_mbb)):
#     s_positions = np.linspace(positions_mbb[i], positions_mbb[i] + length, nb_slices)
#     aperture_slices_names = generate_aperture_slices_names(names_mbb[i], nb_slices)
    
#     for nn, ee, ss in zip(aperture_slices_names, aper_slices_mbb, s_positions):
#         s_positions = np.linspace(positions_mbb[i], positions_mbb[i] + length, nb_slices)
#         x_orbits = get_sagitta_effect_on_aperture(s_positions, length, bending_radius, offset_middle_dipole, s_start_dipole=positions_mbb[0])
#         aperture_slices_names = generate_aperture_slices_names(names_mbb[i], nb_slices)
#         aper_slices = make_sagitta_dipole_apertures(x_orbits, aper_mbb)
        
#         for nn, ee, ss in zip(aperture_slices_names, aper_slices, s_positions):
#             env.elements[nn] = ee.copy()
#             insertions.append(env.place(nn, at=ss))

# line.insert(insertions)

In [7]:
tt_new = line.get_table()

In [8]:
%matplotlib tk
line_thick = xt.Line.from_json('../../injection_lines/injection_thick_approx_ap.json')
plotter_after = apertls.InteractiveAperturePlotter(line, line_thick)

Loading line from dict:   0%|          | 0/7477 [00:00<?, ?it/s]

Done loading line from dict.           


2025-04-18 18:32:56.361 python[77789:6345410] +[IMKClient subclass]: chose IMKClient_Modern


In [9]:
tt_aper = line.get_aperture_table()

Compiling ContextCpu kernels...


ld: warning: duplicate -rpath '/Users/gebruiker/miniforge3/envs/xsuite2/lib' ignored
ld: warning: duplicate -rpath '/Users/gebruiker/miniforge3/envs/xsuite2/lib' ignored


Done compiling ContextCpu kernels.


In [11]:
tt_aper

Table: 84209 rows, 10 cols
name                             s    x_aper_low   x_aper_high x_aper_low_discrete ...
sps$start                        0       -0.0765        0.0755                 nan
begi.10010                       0       -0.0765        0.0755                 nan
qf.10010                         0       -0.0765        0.0755                 nan
drift_0..0                   3.085       -0.0765        0.0755                 nan
veqf.10010.b_aper           3.2504       -0.0765        0.0755             -0.0765
drift_0..1                  3.2504       -0.0765        0.0755                 nan
veba.10030.a_aper           3.2516       -0.0765        0.0755             -0.0765
drift_0..2                  3.2516       -0.0765        0.0755                 nan
mba.10030_entry            3.44499       -0.0765        0.0755                 nan
mba.10030_aper..0::0       3.44499       -0.0765        0.0755             -0.0765
...
drift_1904             6911.11    -0.0764084     0.0

In [10]:
mba_10030 = tt_aper.rows['mba.*']

In [ ]:
for mba in mba_10030.name:
    if hasattr(line[name], 'shift_x'):
        print(line[name].shift_x)

In [18]:
line['mba.10030..0_aper2']

View of LimitRect(min_x=-0.0716, max_x=0.0804, min_y=-0.0193, max_y=0.0193, _sin_rot_s=-999, _cos_rot_s=-999, _shift_x=0, _shift_y=0, _shift_s=0)

In [11]:
mba_10030

Table: 13320 rows, 10 cols
name                             s    x_aper_low   x_aper_high x_aper_low_discrete ...
mba.10030_entry            3.44499       -0.0715        0.0805                 nan
mba.10030_aper..0          3.44499       -0.0765        0.0755             -0.0765
mba.10030..entry_map       3.44499       -0.0715        0.0805                 nan
mba.10030_aper..1          3.44499       -0.0765        0.0755             -0.0765
mba.10030..0_aper2         3.44499       -0.0715        0.0805             -0.0715
mba.10030..0               3.44499       -0.0715        0.0805                 nan
mba.10030_aper..2          4.07099       -0.0765        0.0755             -0.0765
mba.10030..1_aper2         4.07099       -0.0735        0.0785             -0.0735
mba.10030..1               4.07099       -0.0735        0.0785                 nan
mba.10030_aper..3          4.69699       -0.0765        0.0755             -0.0765
...
mba.63590_aper..9          6907.9       -0.0765     

In [23]:
line['mba.10030..0'].name_associated_aperture

In [20]:
line['mba.10030_aper..2'] = line['mba.10030..1_aper2'].copy()

ValueError: Only scalars or references are allowed

In [ ]:
line.remove()

In [30]:
line.particle_ref

Particles(_capacity=np.int64(1), _num_active_particles=np.int64(1), _num_lost_particles=np.int64(0), start_tracking_at_element=np.int64(-1), q0=1, mass0=9.38e+08, t_sim=2.31e-05, _p0c=array([2.592e+10]), _gamma0=array([27.64334239]), _beta0=array([0.99934547]), s=array([0.]), zeta=array([0.]), x=array([0.]), y=array([0.]), px=array([0.]), py=array([0.]), _ptau=array([-2.22044605e-16]), _delta=array([0.]), _rpp=array([1.]), _rvv=array([1.]), chi=array([1.]), charge_ratio=array([1.]), weight=array([1.]), ax=array([0.]), ay=array([0.]), pdg_id=array([2212]), particle_id=array([0]), at_element=array([0]), at_turn=array([0]), state=array([1]), parent_particle_id=array([0]), _rng_s1=array([0], dtype=uint32), _rng_s2=array([0], dtype=uint32), _rng_s3=array([0], dtype=uint32), _rng_s4=array([0], dtype=uint32))

In [22]:
line.slice_thick_elements(slicing_strategies=[xt.Strategy(slicing=xt.Teapot(nb_slices, mode='thick'), name='mb.*'),
                                              xt.Strategy(slicing=xt.Uniform(1, mode='thick'))])

The line already has an associated tracker


Slicing line:   0%|          | 0/15184 [00:00<?, ?it/s]

{'sps$start': ['sps$start'],
 'begi.10010': ['begi.10010'],
 'qf.10010': ['qf.10010_entry',
  'qf.10010_aper..0',
  'qf.10010..entry_map',
  'qf.10010_aper..1',
  'qf.10010..0',
  'qf.10010_aper..2',
  'qf.10010..exit_map',
  'qf.10010_exit'],
 'drift_0..0': ['drift_0..0..0'],
 'veqf.10010.b_aper': ['veqf.10010.b_aper'],
 'drift_0..1': ['drift_0..1..0'],
 'veba.10030.a_aper': ['veba.10030.a_aper'],
 'drift_0..2': ['drift_0..2..0'],
 'mba.10030': ['mba.10030_entry',
  'mba.10030_aper..0',
  'mba.10030..entry_map',
  'mba.10030_aper..1',
  'mba.10030..0',
  'mba.10030_aper..2',
  'mba.10030..exit_map',
  'mba.10030_exit'],
 'drift_1..0': ['drift_1..0..0'],
 'veba.10030.b_aper': ['veba.10030.b_aper'],
 'drift_1..1': ['drift_1..1..0'],
 'veba.10050.a_aper': ['veba.10050.a_aper'],
 'drift_1..2': ['drift_1..2..0'],
 'mba.10050': ['mba.10050_entry',
  'mba.10050_aper..0',
  'mba.10050..entry_map',
  'mba.10050_aper..1',
  'mba.10050..0',
  'mba.10050_aper..2',
  'mba.10050..exit_map',
  'mba.